# 在自定义数据集上训练mattor

1.提供一个新的数据集
2.修改config
3.训练一个新的mattor

## 进入 MMEditing 主目录

In [1]:
import os
os.chdir('mmediting')
os.listdir()

['.git',
 '.circleci',
 '.dele.yml',
 '.dev_scripts',
 '.github',
 '.gitignore',
 '.owners.yml',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmedit',
 'model-index.yml',
 'projects',
 'requirements.txt',
 'requirements',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmedit.egg-info',
 'outputs',
 'data',
 'checkpoints',
 'work_dirs']

## 提供一个新的数据集

在本教程中，我们给出了一个将数据转换成现有数据集格式的例子。

首先，让我们从alphamatting.com 下载唯一可用的开源抠图数据集。

In [2]:
!mkdir -p data/alphamatting/
!curl http://alphamatting.com/datasets/zip/input_training_lowres.zip -o data/alphamatting/input_training_lowres.zip
!curl http://alphamatting.com/datasets/zip/trimap_training_lowres.zip -o data/alphamatting/trimap_training_lowres.zip
!curl http://alphamatting.com/datasets/zip/gt_training_lowres.zip -o data/alphamatting/gt_training_lowres.zip
!unzip -o data/alphamatting/input_training_lowres.zip -d data/alphamatting/merged
!unzip -o data/alphamatting/trimap_training_lowres.zip -d data/alphamatting/trimap
!unzip -o data/alphamatting/gt_training_lowres.zip -d data/alphamatting/alpha

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 17.3M  100 17.3M    0     0  2997k      0  0:00:05  0:00:05 --:--:-- 4626k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  366k  100  366k    0     0   217k      0  0:00:01  0:00:01 --:--:--  216k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2215k  100 2215k    0     0   726k      0  0:00:03  0:00:03 --:--:--  726k
Archive:  data/alphamatting/input_training_lowres.zip
  inflating: data/alphamatting/merged/GT27.png  
  inflating: data/alphamatting/merged/GT01.png  
  inflating: data/alphamatting/merged/GT02.png  
  inflating: data/alphamatting/merged/GT03.png  
  inflating: data/alphamatting/merged/G

然后，我们为训练数据创建注释文件。

In [5]:
# 20个样本进行训练，7个样本进行测试
num_file = 27
num_training = 20  

training_ann = list()
for i in range(num_training):
  ann = dict()
  ann['merged_path'] = f'merged/GT{i+1:02d}.png'
  ann['alpha_path'] = f'alpha/GT{i+1:02d}.png'
  #由于来自alphamatting.com的数据没有合成，我们使用原始图像
  #作为fg 和 bg
  ann['fg_path'] = ann['merged_path']
  ann['bg_path'] = ann['merged_path']
  training_ann.append(ann)

from mmengine import dump
dump(training_ann, './data/alphamatting/training_list.json')

让我们以同样的方式为测试数据创建注释文件。

In [6]:
num_trimap = 1
test_ann = list()
for i in range(num_training, num_file):
  for j in range(num_trimap):
    ann = dict()
    ann['merged_path'] = f'merged/GT{i+1:02d}.png'
    ann['trimap_path'] = f'trimap/Trimap{j+1}/GT{i+1:02d}.png'
    ann['alpha_path'] = f'alpha/GT{i+1:02d}.png'
    test_ann.append(ann)

dump(test_ann, './data/alphamatting/test_list.json')

## 修改config

在下一步中，我们需要修改训练的config。为了加速这个过程，我们使用一个预先训练好的mattor微调一个mattor。

In [10]:
from mmengine import Config
cfg = Config.fromfile('configs/indexnet/indexnet_mobv2_1xb16-78k_comp1k.py')
print(cfg.pretty_text)

dataset_type = 'AdobeComp1kDataset'
data_root = 'data/adobe_composition-1k'
train_dataloader = dict(
    num_workers=8,
    persistent_workers=False,
    sampler=dict(type='InfiniteSampler', shuffle=True),
    dataset=dict(
        type='AdobeComp1kDataset',
        data_root='data/adobe_composition-1k',
        ann_file='training_list.json',
        test_mode=False,
        pipeline=[
            dict(
                type='LoadImageFromFile', key='alpha', color_type='grayscale'),
            dict(type='LoadImageFromFile', key='fg'),
            dict(type='LoadImageFromFile', key='bg'),
            dict(type='LoadImageFromFile', key='merged'),
            dict(type='GenerateTrimapWithDistTransform', dist_thr=20),
            dict(
                type='CropAroundUnknown',
                keys=['alpha', 'merged', 'fg', 'bg', 'trimap'],
                crop_sizes=[320, 480, 640],
                interpolations=[
                    'bicubic', 'bicubic', 'bicubic', 'bicubic', 'nearest'
 

给定一个在Adobe Composition-1k数据集上训练IndexNet模型的config，我们需要修改一些值来使用它在我们刚刚创建的数据集上训练IndexNet。

In [51]:
from mmengine import Config
cfg = Config.fromfile('configs/indexnet/indexnet_mobv2_1xb16-78k_comp1k.py')

from mmengine.runner import set_random_seed

# 修改数据集类型和路径
cfg.train_cfg.type = 'AdobeComp1kDataset'
cfg.train_cfg.ann_file = './data/alphamatting/training_list.json'
cfg.train_cfg.data_prefix = './data/alphamatting/'

cfg.val_cfg.type = 'AdobeComp1kDataset'
cfg.val_cfg.ann_file = './data/alphamatting/test_list.json'
cfg.val_cfg.data_prefix = './data/alphamatting/'

cfg.test_cfg.type = 'AdobeComp1kDataset'
cfg.test_cfg.ann_file = './data/alphamatting/test_list.json'
cfg.test_cfg.data_prefix = './data/alphamatting/'

# 我们可以使用预先训练的IndexNet模型
cfg.model.pretrained = None
cfg.load_from = 'indexnet_mobv2_1x16_78k_comp1k_SAD-45.6_20200618_173817-26dd258d.pth'

# 设置工作目录来保存文件和日志
cfg.work_dir = './tutorial_exps/indexnet'

# 使用小batch size训练
cfg.train_dataloader.sampler = 4
cfg.train_dataloader.workers_per_gpu = 1

#原始学习率(LR)设置batch size为16，使用1个GPU。
#我们将LR降低了4倍，因为我们减少了batch size大小。
cfg.optim_wrapper.optimizer.lr = cfg.optim_wrapper.optimizer.lr / 4
cfg.total_iters = 50
cfg.lr_config = None

# 每20次迭代评估一次
cfg.train_cfg.val_interval = 20

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpus = 1

print(cfg.pretty_text)

dataset_type = 'AdobeComp1kDataset'
data_root = 'data/adobe_composition-1k'
train_dataloader = dict(
    num_workers=8,
    persistent_workers=False,
    sampler=4,
    dataset=dict(
        type='AdobeComp1kDataset',
        data_root='data/adobe_composition-1k',
        ann_file='training_list.json',
        test_mode=False,
        pipeline=[
            dict(
                type='LoadImageFromFile', key='alpha', color_type='grayscale'),
            dict(type='LoadImageFromFile', key='fg'),
            dict(type='LoadImageFromFile', key='bg'),
            dict(type='LoadImageFromFile', key='merged'),
            dict(type='GenerateTrimapWithDistTransform', dist_thr=20),
            dict(
                type='CropAroundUnknown',
                keys=['alpha', 'merged', 'fg', 'bg', 'trimap'],
                crop_sizes=[320, 480, 640],
                interpolations=[
                    'bicubic', 'bicubic', 'bicubic', 'bicubic', 'nearest'
                ]),
            dict(
    

## 训练一个新的mattor

最后，让我们初始化数据集和mattor，然后训练一个新的mattor！

In [57]:
import os.path as osp

from mmedit.datasets import AdobeComp1kDataset
from mmedit.models import BaseEditModel
from mmedit.apis import init_model

import mmcv

# 构建数据集
datasets = [AdobeComp1kDataset(cfg.train_cfg)]

# 构建mattor
model = BaseEditModel(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

# 创建工作目录
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# 训练模型
init_model(model, datasets, cfg, distributed=False, validate=True)

TypeError: expected str, bytes or os.PathLike object, not ConfigDict